In [ ]:
from __future__ import division
import pandas as pd
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import MaxNLocator

from matplotlib.ticker import FormatStrFormatter

import scipy.stats as sstat
import scipy.signal as ssig
import h5py
from mpl_toolkits.mplot3d import Axes3D
import os
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA as sklearnPCA
import re

# import ephys_unit_analysis as ena
import mz_ephys_unit_analysis as mz_ena
import mz_unit_dur_freq as mz_udf

#import resampy
import fnmatch
import seaborn as sns
%matplotlib inline
%load_ext autoreload
%autoreload 2
mpl.rcParams['pdf.fonttype'] = 42 
mpl.rcParams['font.sans-serif']=['Arial', 'Helvetica','Bitstream Vera Sans', 'DejaVu Sans', 'Lucida Grande', 
                                 'Verdana', 'Geneva', 'Lucid', 'Avant Garde', 'sans-serif']  

sns.despine()

In [2]:
# for publication quality plots
def set_pub_plots(pal=sns.blend_palette(["gray","crimson", 'cyan', 'magenta', 'purple'  ],5)):
    sns.set_style("white")
    sns.set_palette(pal)
    sns.set_context("poster", font_scale=1.5, rc={"lines.linewidth": 2.5, "axes.linewidth":2.5, 'figure.facecolor': 'white'}) 
    sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8})
    plt.rcParams['axes.linewidth'] = 2.5

rc_pub={'font.size': 25, 'axes.labelsize': 25, 'legend.fontsize': 25.0, 
        'axes.titlesize': 25, 'xtick.labelsize': 25, 'ytick.labelsize': 25, 
        'axes.linewidth':2.5, 'lines.linewidth': 2.5,
        'xtick.color': 'black', 'ytick.color': 'black', 'axes.edgecolor': 'black',
        'axes.labelcolor':'black','text.color':'black'}
# to restore the defaults, call plt.rcdefaults() 

set_pub_plots()

In [ ]:
pal=sns.blend_palette(['cyan','magenta','grey','crimson'], 4)
sns.palplot(pal)
sns.set_palette(pal)

---

# First, load in the previously saved dataFrame

In [49]:
V1 = pd.read_pickle(r"C:\Users\AChub_Lab\Desktop\zimmer94\Data_Analysis\Multi_brain_regions\V1Hippo_spikes\v1_allrg_df.pkl")
HPC = pd.read_pickle(r"C:\Users\AChub_Lab\Desktop\zimmer94\Data_Analysis\Multi_brain_regions\V1Hippo_spikes\hpc_allrg_df.pkl")

# combine the pre/post df with the novel df
V1HPC_combo = pd.concat([V1, HPC])

In [ ]:
V1HPC_combo.head(2)

In [ ]:
print(V1HPC_combo.stim.unique())
print(V1HPC_combo.r_groups.unique())

# New column that's depth in brain, not on probe

# Histogram of the depth for each r_group

In [ ]:
V1.head(2)

In [ ]:
to_plot = 0
pathway = r'C:\Users\AChub_Lab\Desktop\zimmer94\Data_Analysis\Multi_brain_regions\saved_figs\depth'

for i in np.sort(V1.r_groups.unique()): #make new plot for each r_group from the clustering algorithm above
    f, ax = plt.subplots(1,2, sharex=False, sharey=False, figsize = (15,3))
    ax = ax.ravel()
    # cbar_ax = f.add_axes([.91, .55, .02, .30])
    sns.set_style("ticks")
    
    g1 = sns.heatmap(V1[V1.r_groups==i].pivot_table(index=['cuid'], columns=['times'], values=['zscore']),
                     cmap = 'jet', ax = ax[0], xticklabels=50, yticklabels=False,
                     vmax=5, center=2, vmin=-1, robust=True, cbar_kws=dict(ticks=[-1,0,1,2,3,4,5]))
    ax[0].set(xlabel='Time (s)', ylabel=len(V1[V1.r_groups==i]['cuid'].unique()))
    ax[0].set_xticklabels(np.arange(0,3,0.5), rotation=0)
    ax[0].set_title('group ' + str(i), loc = 'center')
        
    g2 = sns.lineplot(x = 'times', y = 'ztc', estimator = 'mean', data = V1[V1.r_groups==i], 
                      color = 'grey', ax=ax[1], ci=None)
    ax[1].set(xlabel='Time (s)', ylabel='ztc')
    ax[1].set_title('group ' + str (i), loc = 'center')
    ax[1].axhline(y=0, color='k', linestyle="--", linewidth=1.0, alpha=0.75)
    ax[1].xaxis.set_major_locator(MaxNLocator(7))
    ax[1].tick_params(axis='x', labelrotation=0)
    ax[1].set_xticks([0,0.5,1,1.5,2,2.5,3])
    ax[1].yaxis.set_major_locator(MaxNLocator(3))
    
#     if to_plot == 1:
#         fig_title = str('v1_rg'+str(i)+'.pdf')
#         out_title = pathway+'\\'+fig_title
#         plt.savefig(out_title, transparent=True)
    
    plt.show()

In [ ]:
v1_rg = np.sort(V1.r_groups.unique())

for r_group in v1_rg:
    print(r_group)
    

In [ ]:
test_rg = V1[(V1['r_groups']==0)&(V1['times']==0.00)]
test_rg.shape

In [ ]:
test_rg.head()

In [ ]:
test_rg['insert'] = abs(test_rg['depth']-3100)

test_rg.head()

In [ ]:
sns.histplot(data=test_rg, x='insert', binwidth=100)